In [1]:
from typing import cast
from itertools import combinations
import scipy.optimize as opt
from pyscf import gto
from qiskit import QuantumCircuit
from qiskit_aer import AerProvider
from qc_practice import VQE, SSVQE
from qc_practice.ansatz import SP, UCCSD, UpCCGSD, HEA

1. run VQE

In [2]:
mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
vqe = VQE(mol)
vqe.ansatz = SP(depth=1)
vqe.parallel = True
vqe.run()

Computing Hamiltonian...... Done
SCF Electronic Energy: -1.873316479161709
SCF Total Energy:      -1.117349034990280


Starting VQE Calculation

Ansatz: HEA
Optimizer: powell
Iteration: 1
Computing uccsd ansatz..... Done
Building quantum circuit... Done
Measuring energy........... Done
Electronic energy: -1.873030078240010

Iteration: 2
Computing uccsd ansatz..... Done
Building quantum circuit... Done
Measuring energy........... Done
Electronic energy: -1.872421476281400

Iteration: 3
Computing uccsd ansatz..... Done
Building quantum circuit... Done
Measuring energy........... Done
Electronic energy: -1.735517181217253

Iteration: 4
Computing uccsd ansatz..... Done
Building quantum circuit... Done
Measuring energy........... Done
Electronic energy: -1.566211482083210

Iteration: 5
Computing uccsd ansatz..... Done
Building quantum circuit... Done
Measuring energy........... Done
Electronic energy: -1.814747703556842

Iteration: 6
Computing uccsd ansatz..... Done
Building quantum circuit

Singlet_0

2. set qc[0] as the ground state

In [5]:
number_of_states = 2

no = vqe.profile.num_orb
qc = [QuantumCircuit(no*2, no*2) for _ in range(number_of_states)]
ancila = QuantumCircuit(1, 1)

vqe._initialize_circuit(qc[0])
vqe.ansatz.ansatz(qc[0], vqe.profile, vqe.profile.coeff)


coeff = vqe.ansatz.generate_coeff(vqe.profile)
vqe._initialize_circuit(qc[1])
vqe.ansatz.ansatz(qc[1], vqe.profile, coeff)

3. measuring overlap

In [7]:
def measure_overlap(qc1, qc2):
    no = vqe.profile.num_orb
    ancila = QuantumCircuit(1, 1)
    qc_total = ancila.tensor(qc1).tensor(qc2)
    qc_total.h(0)
    for i in range(1, no*2+1):
        qc_total.ccx(i+no*2, i, 0)
    qc_total.h(0)

    backend = AerProvider().get_backend('qasm_simulator')
    result = cast(dict[str, float],
                backend.run(qc_total, shots=10000).result().get_counts())
    print(result)

measure_overlap(qc[0], qc[1])

/tmp/ipykernel_12656/6180801.py:10: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = AerProvider().get_backend('qasm_simulator')


QiskitError: 'No counts for experiment "0"'

4. make cost function

4. run optimizer